In [ ]:

#pivot_df = pd.read_csv('synthetic_dataset_50_k.csv')
#pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)

#pivot_df = pd.read_csv('top_fifty_motes_temperatures.csv')
pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)
pivot_df = pivot_df.head(10000)

penalty = 1.0  #  penalty for whittle index
num_runs = 3 # Number of simulation runs
M = 5 # Number of nodes to poll at each time step
num_nodes = 50  # Fixed number of nodes
beta_1 = 0.3 # dEWMA parameter for state value
beta_2 = 0.3 # dEWMA parameter for rate of change

# Function to simulate Round Robin scheduling
# Example usage
num_nodes = 50

L = 500000  # Maximum time steps a node can go without being polled


transmission_prob = 1.0  # High transmission probability for simplicity
transmission_probs = np.full(num_nodes, transmission_prob)
# Initialize process and measurement noise covariance (scalars)
Q = 0.0001  # Process noise covariance (scalar)
R = 0.01    # Measurement noise covariance (scalar)
theta = 0.1  # Threshold for polling decision


def run_simulation_round_robin_packets(transmission_probs, num_nodes, M):
    total_packets_rr = 0  # Track the total number of successfully transmitted packets

    last_update_rr = np.zeros(num_nodes)  # Last update time for RR

    def transmission_success(prob):
        return np.random.rand() < prob

    for t in range(1, len(pivot_df)):
        # Poll M nodes in cyclic order (shift to 1-based indexing)
        nodes_rr = [(t - 1 + i) % num_nodes + 1 for i in range(M)]  # Poll M nodes in cyclic order
        for node_rr in nodes_rr:
            if transmission_success(transmission_probs[node_rr - 1]):
                total_packets_rr += 1  # Count successful transmission

    return total_packets_rr
# Function to simulate Whittle Index AoI and count total transmitted packets
def run_simulation_whittle_aoi_packets(transmission_probs, num_nodes, M):
    total_packets_whittle = 0  # Track the total number of successfully transmitted packets

    last_update_whittle = np.zeros(num_nodes)

    def transmission_success(prob):
        return np.random.rand() < prob

    for t in range(1, len(pivot_df)):
        whittle_indices = {}

        # Compute Whittle indices for each node (shift to 1-based indexing)
        for node in range(1, num_nodes + 1):
            delta_t_dynamic = t - last_update_whittle[node - 1]  # Time since last update (AoI)
            current_aoi = delta_t_dynamic
            future_aoi_passive = delta_t_dynamic + 1
            future_aoi_active = 0 if transmission_success(transmission_probs[node - 1]) else future_aoi_passive

            q_passive = current_aoi + future_aoi_passive
            q_active = current_aoi + future_aoi_active + 0  # No additional penalty for active action
            whittle_indices[node] = q_passive - q_active

        # Poll top M nodes with the highest Whittle index
        nodes_to_poll = [node for node, index in whittle_indices.items() if index >= 0]
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=lambda node: whittle_indices[node], reverse=True)[:M]  # Top M nodes

        for node_whittle in nodes_to_poll:
            if transmission_success(transmission_probs[node_whittle - 1]):
                total_packets_whittle += 1  # Count successful transmission

    return total_packets_whittle
def run_simulation_whittle_aoii_packets(pivot_df, transmission_probs, num_nodes, M=5, L=5000):
    total_packets_whittle_aoii = 0  # Track the total number of successfully transmitted packets
    aoii_history = {f'mote{i}': [] for i in range(1, num_nodes + 1)}  # History of AoII for each node
    whittle_indices = {}
    
    # Initialize last update time and time since last poll for each node
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}
    time_since_last_poll = {i: 0 for i in range(1, num_nodes + 1)}  # Track how long it's been since a node was last polled

    # Initialize node and sink states
    state_node = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}  # Node state
    state_sink = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}  # Sink state (constant)

    # Run the simulation
    for t in range(len(pivot_df)):
        # Update the state of each node using dEWMA
        for mote in range(1, num_nodes + 1):
            column_name = f'mote{mote}'
            measured_value = pivot_df.loc[t, column_name]
            last_state_value, last_rate_of_change = state_node[column_name]
            state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)

        # Update the time since each node was last polled
        for mote in range(1, num_nodes + 1):
            time_since_last_poll[mote] = t - last_update_times[mote]

        # Compute Whittle index for each node
        for mote in range(1, num_nodes + 1):
            column_name = f'mote{mote}'
            delta_t_dynamic = t - last_update_times[mote]  # Time since last update

            # Calculate the current AoII based on the rate of change of information (drift)
            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            future_aoii_passive = predict_node_aoii_from_sink(state_sink[f'mote{mote}'], delta_t_dynamic + 1)
            future_aoii_active = 0 if np.random.rand() < transmission_probs[mote - 1] else future_aoii_passive

            # Whittle index: compare passive and active AoII penalties
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + penalty
            whittle_indices[mote] = q_passive - q_active

        # Select the top M nodes with the highest Whittle index
        nodes_to_poll = [mote for mote in whittle_indices if whittle_indices[mote] >= 0]

        # If we have more nodes than M, select the top M nodes
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

        # Check for nodes that haven't been polled for more than L time steps and apply the constraint
        for mote in range(1, num_nodes + 1):
            if (t - last_update_times[mote]) > L and mote not in nodes_to_poll:
                # If there are any nodes currently selected for polling, find the node with the lowest Whittle index
                if len(nodes_to_poll) > 0:
                    lowest_whittle_node = min(nodes_to_poll, key=lambda x: whittle_indices[x])
                    nodes_to_poll.remove(lowest_whittle_node)  # Remove the lowest Whittle index node
                nodes_to_poll.append(mote)  # Add the neglected node

        # Poll the selected nodes and update their state
        for mote in nodes_to_poll:
            column_name = f'mote{mote}'
            measured_value = pivot_df.loc[t, column_name]
            last_state_value, last_rate_of_change = state_node[column_name]
            delta_t_dynamic = t - last_update_times[mote]  # Time since last update

            # Predict AoII for the polled node
            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            aoii_history[f'mote{mote}'].append(current_aoii)  # Record the AoII for the node
            state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)
            state_sink[column_name] = state_node[column_name]

            # If transmission is successful, count the transmitted packet
            if np.random.rand() < transmission_probs[mote - 1]:
                total_packets_whittle_aoii += 1  # Count successful transmission

            # Reset the update time for the polled node
            last_update_times[mote] = t

    return total_packets_whittle_aoii

def run_simulation_f_w_aoii_packets(pivot_df, transmission_probs, num_nodes, M=5, L=1000):
    total_packets_whittle_aoii = 0  # Track the total number of successfully transmitted packets
    aoii_history = {f'mote{i}': [] for i in range(1, num_nodes + 1)}  # History of AoII for each node
    whittle_indices = {}
    
    # Initialize last update time and time since last poll for each node
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}
    time_since_last_poll = {i: 0 for i in range(1, num_nodes + 1)}  # Track how long it's been since a node was last polled

    # Initialize node and sink states
    state_node = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}  # Node state
    state_sink = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}  # Sink state (constant)

    # Run the simulation
    for t in range(len(pivot_df)):
        # Update the state of each node using dEWMA
        for mote in range(1, num_nodes + 1):
            column_name = f'mote{mote}'
            measured_value = pivot_df.loc[t, column_name]
            last_state_value, last_rate_of_change = state_node[column_name]
            state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)

        # Update the time since each node was last polled
        for mote in range(1, num_nodes + 1):
            time_since_last_poll[mote] = t - last_update_times[mote]

        # Compute Whittle index for each node
        for mote in range(1, num_nodes + 1):
            column_name = f'mote{mote}'
            delta_t_dynamic = t - last_update_times[mote]  # Time since last update

            # Calculate the current AoII based on the rate of change of information (drift)
            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            future_aoii_passive = predict_node_aoii_from_sink(state_sink[f'mote{mote}'], delta_t_dynamic + 1)
            future_aoii_active = 0 if np.random.rand() < transmission_probs[mote - 1] else future_aoii_passive

            # Whittle index: compare passive and active AoII penalties
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + penalty
            whittle_indices[mote] = q_passive - q_active

        # Select the top M nodes with the highest Whittle index
        nodes_to_poll = [mote for mote in whittle_indices if whittle_indices[mote] >= 0]
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

        # Identify nodes that exceed the L constraint
        nodes_exceeding_L = [mote for mote in range(1, num_nodes + 1) if (t - last_update_times[mote]) > L]

        # Replace nodes to meet constraints if necessary
        if nodes_exceeding_L:
            if len(nodes_exceeding_L) <= M:
                # Remove the lowest-ranked nodes from Whittle policy to add L-violating nodes
                nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M - len(nodes_exceeding_L)]
                nodes_to_poll.extend(nodes_exceeding_L)
            else:
                # Only poll top M nodes that are violating the L constraint
                nodes_to_poll = sorted(nodes_exceeding_L, key=lambda x: time_since_last_poll[x], reverse=True)[:M]

        # Poll the selected nodes and update their state
        for mote in nodes_to_poll:
            column_name = f'mote{mote}'
            measured_value = pivot_df.loc[t, column_name]
            last_state_value, last_rate_of_change = state_node[column_name]
            delta_t_dynamic = t - last_update_times[mote]  # Time since last update

            # Predict AoII for the polled node
            current_aoii = predict_node_aoii_from_sink(state_sink[column_name], delta_t_dynamic)
            aoii_history[f'mote{mote}'].append(current_aoii)  # Record the AoII for the node
            state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)
            state_sink[column_name] = state_node[column_name]

            # If transmission is successful, count the transmitted packet
            if np.random.rand() < transmission_probs[mote - 1]:
                total_packets_whittle_aoii += 1  # Count successful transmission

            # Reset the update time for the polled node
            last_update_times[mote] = t

    return total_packets_whittle_aoii


def run_simulation_kalman_packets(pivot_df, num_nodes, num_nodes_to_poll, L=1000, theta=0.5):
    """Runs the simulation using Kalman filter (1D) and returns the total number of transmitted packets."""
    
    # Initialize state estimates and covariance matrices (scalars)
    state_estimates = {f'mote{i}': 20.0 for i in range(1, num_nodes + 1)}  # Initial state estimates
    P = {f'mote{i}': 0.1 for i in range(1, num_nodes + 1)}  # Initial covariance matrices (scalars)
    last_update_times = {f'mote{i}': 0 for i in range(1, num_nodes + 1)}  # Time when the last measurement was received
    total_packets = 0  # Track total number of successfully transmitted packets

    for idx, row in pivot_df.iterrows():
        current_time_step = idx

        # Calculate covariance traces (which are scalars here)
        covariance_traces = {mote: P[f'mote{mote}'] for mote in range(1, num_nodes + 1)}
        sorted_motes_by_trace = sorted(covariance_traces, key=covariance_traces.get, reverse=True)
        nodes_to_poll = set(sorted_motes_by_trace[:num_nodes_to_poll])

        # Fairness constraint: Poll nodes that have not been updated in L time steps
        for mote in range(1, num_nodes + 1):
            if (current_time_step - last_update_times[f'mote{mote}']) > L and mote not in nodes_to_poll:
                lowest_trace_node = sorted_motes_by_trace[-1]
                if lowest_trace_node in nodes_to_poll:
                    nodes_to_poll.remove(lowest_trace_node)
                nodes_to_poll.add(mote)

        # Process the selected nodes
        for mote in nodes_to_poll:
            column_name = f'mote{mote}'
            if column_name in row:
                measured_value = row[column_name]  # New measurement received
                previous_state = state_estimates[column_name]  # Use last state estimate
                previous_P = P[column_name]  # Use last covariance matrix (scalar)
                delta_t = max(current_time_step - last_update_times[column_name], 1)  # Time since last measurement

                # Prediction step
                xp = previous_state  # Predict future state (since it's 1D, xp is just the last state)
                Pp = previous_P + Q  # Predict future covariance (scalar), assuming Q=0.1

                # If the difference exceeds the threshold, transmit the packet (successful update)
                if abs(xp - measured_value) > theta :
                    # Update step
                    K = Pp / (Pp + R)  # Kalman gain (scalar), assuming R=0.1
                    x_hat = xp + K * (measured_value - xp)  # Update state estimate using new measurement (scalar)
                    P_hat = (1 - K) * Pp  # Update covariance (scalar)
                    
                    state_estimates[column_name] = x_hat  # Update state estimate
                    P[column_name] = P_hat  # Update covariance
                    total_packets += 1  # Increment total packets for successful transmissions
                    last_update_times[column_name] = current_time_step  # Update the last time of measurement

    return total_packets  # Return the total number of successfully transmitted packets

def run_simulation_top_k_query_packets(pivot_df, M, transmission_prob=0.9):
    """Runs the simulation using Top-k query technique and returns the total number of transmitted packets."""
    total_packets = 0  # Track the total number of successfully transmitted packets

    for idx, row in pivot_df.iterrows():
        # Drop any non-numeric columns if present, such as 'SN'
        row = row.drop(['SN'], errors='ignore')

        # Get the indices of the top `num_nodes_to_poll` values based on argmax
        top_k_indices = row.values.argsort()[-M:][::-1]  # Get top-k node indices (zero-based)

        # Perform actions for nodes that are being polled
        for mote in top_k_indices:
            # Simulate transmission success based on the transmission probability
            if np.random.rand() < transmission_prob:
                total_packets += 1  # Increment total packets for successful transmissions

    return total_packets  # Return the total number of successfully transmitted packets


total_packets_whittle_aoii = run_simulation_whittle_aoii_packets(pivot_df, transmission_probs, num_nodes, M)
total_packets_f_w_aoii = run_simulation_f_w_aoii_packets(pivot_df, transmission_probs, num_nodes, M)
total_packets_rr = run_simulation_round_robin_packets(transmission_probs, num_nodes, M)
total_packets_whittle = run_simulation_whittle_aoi_packets(transmission_probs, num_nodes, M)
total_packets_kalman = run_simulation_kalman_packets(pivot_df, num_nodes, M, L, theta)
# Run the simulation and get the total packets transmitted
total_packets_top_k = run_simulation_top_k_query_packets(pivot_df, M, transmission_prob)

print(f"Total Packets Transmitted (Round Robin): {total_packets_rr}")
print(f"Total Packets Transmitted (Whittle AoI): {total_packets_whittle}")
print(f"Total Packets Transmitted (Whittle AoII): {total_packets_whittle_aoii}")
print(f"Total Packets Transmitted (F_W_AoII): {total_packets_f_w_aoii}")
print(f"Total Packets Transmitted (Kalman Filter): {total_packets_kalman}")
print(f"Total Packets Transmitted (Top-k Query): {total_packets_top_k}")